In [5]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn


class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx].values.astype('float32')
        return torch.tensor(sample)

dataset = CustomDataset('sample_10k.csv')

In [6]:
batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [7]:
input_dim = 30 
latent_size = 64
hidden_size = 256  

In [8]:
G = nn.Sequential(
    nn.Linear(latent_size, 256),  
    nn.LeakyReLU(0.2),
    nn.Linear(256, 512),           
    nn.LeakyReLU(0.2),
    nn.Linear(512, 1024),         
    nn.LeakyReLU(0.2),
    nn.BatchNorm1d(1024),         
    nn.Linear(1024, input_dim),   
    nn.Tanh()                     
)


In [9]:
D = nn.Sequential(
    nn.Linear(input_dim, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
)

In [10]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0001)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [ ]:
import torch.nn as nn
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

G.to(device)
D.to(device)


def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

def train_discriminator(real_data):

    real_labels = torch.ones(batch_size, 1).to(device)
    fake_labels = torch.zeros(batch_size, 1).to(device)
    
    outputs = D(real_data)
    d_loss_real = criterion(outputs, real_labels)
    real_score = outputs
    
    z = torch.randn(batch_size, latent_size).to(device)
    fake_data = G(z)
    outputs = D(fake_data)
    d_loss_fake = criterion(outputs, fake_labels)
    fake_score = outputs
    
    d_loss = d_loss_real + d_loss_fake
    reset_grad()
    d_loss.backward()
    d_optimizer.step()
    
    return d_loss, real_score, fake_score

def train_generator():
    z = torch.randn(batch_size, latent_size).to(device)
    fake_data = G(z)
    labels = torch.ones(batch_size, 1).to(device) 
    g_loss = criterion(D(fake_data), labels)
    
    reset_grad()
    g_loss.backward()
    g_optimizer.step()
    
    return g_loss, fake_data

num_epochs = 10
total_step = len(data_loader)

for epoch in range(num_epochs):
    for i, real_data in enumerate(data_loader):
        real_data = real_data.to(device)
        
        real_data = real_data.reshape(batch_size, -1)
        
        d_loss, real_score, fake_score = train_discriminator(real_data)
        
        g_loss, fake_data = train_generator()
        
        if (i + 1) % 200 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Step [{i+1}/{total_step}], d_loss: {d_loss.item():.4f}, '
                  f'g_loss: {g_loss.item():.4f}, D(x): {real_score.mean().item():.2f}, D(G(z)): {fake_score.mean().item():.2f}')

    if (epoch + 1) % 10 == 0:
        z = torch.randn(batch_size, latent_size).to(device)
        fake_data = G(z)
        print(f"Epoch {epoch + 1}: Üretilen örnek veri - {fake_data[0]}")
